In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt

In [5]:
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import *
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator 
import keras.backend as k

In [ ]:
Img_path = 
Mask_path = 


In [ ]:
# List of images
all_images =[x for x in sorted(os.listdir(Img_path)) if x[-4:]== '.tif']
all_masks = [x for x in sorted(os.listdir(Mask_path)) if x[-4:] == '.tif']
print(len(all_images))
print(len(all_masks))

In [6]:
# Resize and Normalize images
X_data = np.empty((len(all_images),Img_height, Img_width), dtypes='float32')
print(X_data.shape)
for i, name in enumerate(all_images):
    im = cv2.imread(Img_path+name, cv2.IMREAD_UNCHANGED).astype('int16').astype('float32')
    im = cv2.resize(im, dsize = (Img_width, Img_height), Interpolation=cv2.INTER_LANC20S4)
    img = (im - np.min(im))/ (np.max(im)- np.min(im))
    X_data[i] = img

Y_data = np.empty(len(all_images), Img_height, Img_width), dtype='float32')
print(Y_data.shape)
for i, name in enumerate(all_images):
    im = cv2.imread(Mask_path+name, cv2.IMREAD_UNCHANGED).astype('int16').astype('float32')
    im = cv2.resize(im, dsize = (Img_width, Img_height), Interpolation=cv2.INTER_NEAREST)
    X_data[i] = img

SyntaxError: unexpected EOF while parsing (<ipython-input-6-8e3b346cb37f>, line 3)

In [ ]:
# Display images
fig, ax = plt.subplots(1, 2, figsize=(8,4))
ax[0].imshow(X_data[0], cmap='gray')
ax[1].imshow(Y_data[0], cmap='gray')
plt.show()

In [ ]:
# Train test split
X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size=0.5)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

In [ ]:
# Model building
input_layer = Input(shape=X_train.shape[1:])
c1 = Conv2D(filters=8, kernel_size=(3,3), activation='relu', padding='same')(input_layer)
l = MaxPool2D(strides=(2,2))(c1)
c2 = Conv2D(filters=16, kernel_size=(3,3), activation='relu', padding='same')(l)
l = MaxPool2D(strides=(2,2))(c2)
c3 = Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same')
l = MaxPoll2D(strides=(2,2))(c3)
c4 = Conv2D(filters=64, kernel_size=(1,1), activation='relu', padding='same')
l = concatenate([UpSampling2D(size=(2,2))(c4), c3], axis=-1)
l = Conv2D(filters=64, kernel_size=(2,2), activation='relu', padding='same')(l)
l = concatenate([UpSampling2D(size=(2,2))(l), c2], axis=1)
l = Conv2D(filters=32, kernel_size=(2,2), activation='relu', padding='same')(l)
l = concatenate([Upsampling2D(size=(2,2)),(l), (c1)], axis=-1)
l = Conv2D(filters=16, kernel_size=(2,2), activation='relu', padding='same')(l)
l = Conv2D(filters = 64, kernel_size=(1,1), activation='relu')(l)
l = Droput(0.5)(1)
output_layer = Conv2D(filters = 1, kernel_size=(1,1), actvation='sigmoid')(l)
model = Model(input, output_layer)

In [ ]:
# ImageDataGenerator
def my_generator(X_train, Y_train, batch_size):
    data_generator = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    rotation_range=10,
    zoom_range=0.1).flow(Y_train, Y_train, batch_size, seed=seed)
    
    mask_generator = ImageDataGenerator(
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    rotation_range=10,
    zoom_range=0.1).flow(Y_train, Y_train, batch_size, seed=Seed)
    
    while True:
        x_batch = data_generator.next()
        y_batch = data_generator.next()
        yield x_batch, y_batch
        

In [ ]:
# Display Images after using ImageDataGenerator
image_batch, mask_batch = next(my_generator(X_train, Y_train, 8))
fig, ax = plt.subplots(8, 2, figsize=(8, 20))
for i in range(8):
    ax[i,0].imshow(image_batch[i, :, :, 0])
    ax[i,0].imshow(mask_batch[i, :, :, 0])
plt.show()